### Selenium Scraping

##### Gmarket 베스트 셀러 데이터 스크래핑 
- 전체 200개중 상위 10개 출력
- pandas 데이터를 분석할수 있는 python 라이브러리
    - pip3 install pandas
    - 데이터 프레임 생성 : df = pd.DataFrame(columns=["A","B"])
    - 데이터 추가 : df.loc[len(df)] = [{"A":1,"B":2}, {"A":3,"B":4}]

In [1]:
from selenium import webdriver
import pandas as pd

In [2]:
def get_gmarket_bestsellers(number=10):
    
    def to_int(price):
        return int(price.replace(",","").replace("원",""))
    
    driver =  webdriver.Chrome()
    driver.get('http://corners.gmarket.co.kr/Bestsellers')

    bestlist_selector = driver.find_elements_by_css_selector("#gBestWrap .best-list")[1]
    items = bestlist_selector.find_elements_by_css_selector("li")[:number]
    
    df = pd.DataFrame(columns=["상품명","원가","판매가","이미지","상품링크"])
    
    for item in items:

        idx = item.find_element_by_css_selector("p").text # 순번
        img_link = item.find_element_by_css_selector("img").get_attribute("src") # 이미지 URL
        item_link = item.find_element_by_css_selector(".itemname").get_attribute("href") # 아이템 URL
        content = item.find_element_by_css_selector(".itemname").text # 아이템 텍스트
        o_price = item.find_element_by_css_selector(".item_price .o-price").text # 오리지날 가격
        s_price = item.find_element_by_css_selector(".item_price .s-price").text.split(" ")[0] # 판매 가격
        o_price = o_price if o_price else s_price # 원가 데이터가 없는경우 판매가를 대입
        
        data = {"상품명": content,
                "원가": to_int(o_price), 
                "판매가": to_int(s_price),
                "이미지": img_link, 
                "상품링크": item_link, }
        
        # Pandas 데이터 입력
        df.loc[len(df)] = data
        
    driver.close()
    return df

In [3]:
%%time
result = get_gmarket_bestsellers(200)
result.tail()

,상품명,원가,판매가,이미지,상품링크
195,[삼성전자][G]삼성 TDS신가열 RE-C21KB 21L 전자레인지,90400,85880,http://gdimg.gmarket.co.kr/836058599/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
196,[삼성전자]인증점P 17년신형 진공청소기 VC33M2110LP 당일출고,103000,87900,http://gdimg.gmarket.co.kr/916901271/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
197,비앤티박스 택배박스/착한가격/당일퀵배송/최고품질/,35000,31700,http://gdimg.gmarket.co.kr/293963256/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
198,MAANGE 프로페셔널 메이크업 브러쉬 22종 세트-2color,29900,12900,http://gdimg.gmarket.co.kr/1156619333/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...
199,[소프시스]소프시스 철제5단서랍,36900,28900,http://gdimg.gmarket.co.kr/1207302715/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...


In [4]:
# Pandas로 할인율 계산
result["할인율"] = 100 - (result["판매가"] / result["원가"]) * 100
result["할인율"] = result["할인율"].astype(dtype="float64")
result["할인율"] = round(result["할인율"], 2)
result = result.sort_values("할인율", ascending=False).reset_index().rename(columns={"index": "rank"})
result.head()

,rank,상품명,원가,판매가,이미지,상품링크,할인율
0,124,[다샵]벽걸이 스탠드 에어컨커버 제습기/선풍기커버,26900,6500,http://gdimg.gmarket.co.kr/509405582/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,75.84
1,0,리바이스키즈 여름티셔츠/반팔/반바지/연휴특가,35000,10500,http://gdimg.gmarket.co.kr/1139276022/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
2,19,[슈가든]봄신상 여성 구두 힐 로퍼 단화 운동화 슬립온 종합,33000,9900,http://image.gmarket.co.kr/challenge/neo_image...,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
3,126,[또래친구]2018/상하복/운동복/체육복/기능성/주니어세트,53000,15900,http://gdimg.gmarket.co.kr/706563579/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00
4,113,(키즈꼬모) 봄 상하복 6900 / 아동복 / 등원복,23000,6900,http://gdimg.gmarket.co.kr/1232338881/still/300,http://item2.gmarket.co.kr/Item/DetailView/Ite...,70.00


In [5]:
# utf-8로 인코딩한 csv 파일로 저장
result.to_csv("gmarket.csv", encoding="utf-8")

##### Naver 메일 데이터 스크래핑
- 네이버 메일에 로그인한후 데이터 가져오기

In [6]:
from selenium import webdriver
import pickle
import time

In [13]:
# 패스워드 저장하기
# pw = ""
# file = open("pw.pickle", "wb")
# pickle.dump(pw, file)

In [8]:
main_url = "http://naver.com"
mail_url = "http://mail.naver.com"

In [9]:
# 브라우져 오픈 및 메인 사이트 이동
driver = webdriver.Chrome()
driver.get(main_url)

In [14]:
# 아이디 패스워드 입력
pw = pickle.load(open("pw.pickle", "rb")) # pw 변수에 본인의 패스워드 입력
driver.find_element_by_css_selector( "#id" ).send_keys( "1221pdj" )
driver.find_element_by_css_selector( "#pw" ).send_keys( pw )

In [15]:
# 로그인 버튼 클릭
driver.find_element_by_css_selector("span.btn_login input[type=submit]").click()

In [16]:
# 메일 페이지로 이동
driver.get(mail_url)

In [17]:
# 메일 갯수
total_mail = driver.find_element_by_css_selector("#headTotalNum").text
total_mail = int(total_mail)
total_mail

30

In [18]:
# 안읽은 메일 갯수
not_read_mail = driver.find_element_by_css_selector("#headUnreadNum").text
not_read_mail = int(not_read_mail)
not_read_mail

25

In [19]:
# 메일 제목, 보낸사람 이름 가져오기
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  30


,제목,보낸사람,메일링크
25,SMTP Send File 테스트(제목),utmostar0201@gmai..,https://mail.naver.com/read/popup/?nMailId=11887
26,SMTP Send File 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11886
27,SMTP Send HTML 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11885
28,SMTP Send Text 테스트(제목),junkoo0101@gmail.co..,https://mail.naver.com/read/popup/?nMailId=11884
29,SMTP Send Text 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11883


In [21]:
# marked 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li.mark")
print("전체 메일수 : ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

전체 메일수 :  3


,제목,보낸사람,메일링크
0,"종로구에 있는 사람들이 ""Watch Boston Dynamics’ SpotMini ...",트위터,https://mail.naver.com/read/popup/?nMailId=11911
1,"문성근 님이 트윗함: 너무 오래 걸렸다. '친일행위' 인촌 김성수, 56년만에 건국...",트위터,https://mail.naver.com/read/popup/?nMailId=11910
2,"종로구에 있는 사람들이 ""People who think about leaving r...",트위터,https://mail.naver.com/read/popup/?nMailId=11907


In [22]:
# 읽은 메일
mails = driver.find_elements_by_css_selector(".mailList.sender_context > li:not(.notRead)")
print("total mails: ", len(mails))
df = pd.DataFrame(columns=["제목","보낸사람","메일링크"])
for idx, mail in enumerate(mails):
    title = mail.find_element_by_css_selector(".mail_title").text
    name = mail.find_element_by_css_selector(".name").text
    link = mail.find_element_by_css_selector(".subject > a").get_attribute("href")
    df.loc[len(df)] = { "제목":title, "보낸사람":name, "메일링크":link }    
df.tail()

total mails:  5


,제목,보낸사람,메일링크
0,Google SMTP로 메일 보내기,radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11893
1,Google SMTP로 메일 보내기,radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11890
2,SMTP Send File 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11886
3,SMTP Send HTML 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11885
4,SMTP Send Text 테스트(제목),radajin1224@gmail.c..,https://mail.naver.com/read/popup/?nMailId=11883


In [23]:
# 로그아웃
logout_url = driver.find_element_by_css_selector(".gnb_btn_login").get_attribute("href")
driver.get(logout_url)

In [24]:
# 브라우져 닫기
driver.close()